In [93]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import re
import stanza
stanza.download('en')
stanza_nlp = stanza.Pipeline('en')

2022-05-11 13:04:35 INFO: Downloading default packages for language: en (English)...
2022-05-11 13:04:36 INFO: File exists: /Users/user/stanza_resources/en/default.zip
2022-05-11 13:04:39 INFO: Finished downloading models and saved to /Users/user/stanza_resources.


2022-05-11 13:04:41 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-05-11 13:04:41 INFO: Use device: cpu
2022-05-11 13:04:41 INFO: Loading: tokenize
2022-05-11 13:04:41 INFO: Loading: pos
2022-05-11 13:04:41 INFO: Loading: lemma
2022-05-11 13:04:41 INFO: Loading: depparse
2022-05-11 13:04:41 INFO: Loading: sentiment
2022-05-11 13:04:41 INFO: Loading: constituency
2022-05-11 13:04:41 INFO: Loading: ner
2022-05-11 13:04:42 INFO: Done loading processors!


In [94]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /Users/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/user/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [217]:
from textblob import TextBlob
txt = "The Sound Quality is great but the battery life is very bad, and also what is with the shitty phone size.".lower()
wiki = TextBlob(txt)

In [218]:
def find_noun_phrases_boundaries(pos):
    results = []
    end = -1
    for i in range(len(pos)):
        if pos[i] != 'NN':
            if i != end + 1:
                results.append((end + 1, i))                
            end = i
    if i != end + 1:
        results.append((end + 1, i + 1))                
    return results

In [219]:
tags = [tag[1] for tag in wiki.pos_tags]
noun_phrases_boundaries = find_noun_phrases_boundaries(tags)
noun_phrases = [wiki.words[i:j] for i, j in noun_phrases_boundaries]
noun_phrases

[WordList(['sound', 'quality']),
 WordList(['battery', 'life']),
 WordList(['phone', 'size'])]

In [220]:
for noun_phrase in noun_phrases:
    wiki = TextBlob(re.sub(' '.join(noun_phrase), ''.join(noun_phrase), wiki.string))
    
print(wiki)

the soundquality is great but the batterylife is very bad, and also what is with the shitty phonesize.


In [221]:
import spacy
from spacy import displacy

nlp = spacy.load('en_core_web_sm')
doc = nlp(wiki.string)

In [222]:
selected_tokens = []
for token in doc:
    if token.pos_ in ['NOUN']:
        selected_tokens.append(token.text)
selected_tokens

['soundquality', 'batterylife', 'phonesize']

In [225]:
stanza_doc = stanza_nlp(wiki.string)
deps = []
relations_of_interest = ['nsubj', 'amod']
for dep in stanza_doc.sentences[0].dependencies:
    # print(dep[0], dep[1], dep[2])
    # print('*' * 10)
    if dep[1] in relations_of_interest:
        if dep[0].text in selected_tokens:
            deps.append(((dep[0].text, dep[0].xpos), (dep[2].text, dep[2].xpos)))
        else:
            deps.append(((dep[2].text, dep[2].xpos), (dep[0].text, dep[0].xpos)))

In [227]:
deps

[(('soundquality', 'NN'), ('great', 'JJ')),
 (('batterylife', 'NN'), ('bad', 'JJ')),
 (('phonesize', 'NN'), ('shitty', 'JJ'))]

In [226]:
displacy.render(doc, style='dep')